In [ ]:
from output_evaluator import TransactionEvaluator
from networkx.algorithms import isomorphism
import networkx as nx
import numpy as np
import json
import random
import time
import pandas as pd

from collections import defaultdict

In [ ]:
def compute_frequency_and_success_rate(Q, full_transaction_history):
    """
    计算查询图 Q 中每个顶点和边的交易频率和成功率

    参数：
    - Q: 查询图（networkx.Graph）
    - full_transaction_history: List of transaction records，每个元素为 dict，含有：
        {
            'graph': G,            # networkx.Graph
            'price': float,
            'success': bool
        }

    返回：
    - vertex_freq: dict[node] = frequency
    - vertex_success: dict[node] = success rate
    - edge_freq: dict[edge] = frequency
    - edge_success: dict[edge] = success rate
    """
    vertex_freq = defaultdict(int)
    vertex_success = defaultdict(int)
    edge_freq = defaultdict(int)
    edge_success = defaultdict(int)

    # 将 Q 的节点和边集合取出
    query_nodes = set(Q.nodes())
    query_edges = set(Q.edges())

    # 遍历所有历史交易
    for record in full_transaction_history:
        G = record['graph']
        success = record['success']

        # 统计与 Q 交集部分的节点
        matched_nodes = set(G.nodes()).intersection(query_nodes)
        for node in matched_nodes:
            vertex_freq[node] += 1
            if success:
                vertex_success[node] += 1

        # 统计与 Q 交集部分的边（无向图考虑无向等价）
        matched_edges = set()
        for (u, v) in G.edges():
            if (u in query_nodes) and (v in query_nodes):
                if (u, v) in query_edges or (v, u) in query_edges:
                    matched_edges.add((u, v))

        for (u, v) in matched_edges:
            edge_freq[(u, v)] += 1
            if success:
                edge_success[(u, v)] += 1

    # 计算成功率
    vertex_success_rate = {node: vertex_success[node] / vertex_freq[node]
                           for node in vertex_freq}
    edge_success_rate = {edge: edge_success[edge] / edge_freq[edge]
                         for edge in edge_freq}

    return vertex_freq, vertex_success_rate, edge_freq, edge_success_rate

In [ ]:
def get_centrality_for_query(Q, centrality_file):
    """
    从中心性文件中提取图 Q 中顶点和边的中心性

    参数：
    - Q: 查询图（networkx.Graph）
    - centrality_file: 中心性记录文件（pickle 格式，包含 vertex_centrality 和 edge_centrality）

    返回：
    - vertex_centrality_q: dict[node] = centrality 值
    - edge_centrality_q: dict[(u, v)] = centrality 值
    """
    with open(centrality_file, 'rb') as f:
        vertex_centrality_all, edge_centrality_all = pickle.load(f)

    vertex_centrality_q = {}
    for node in Q.nodes():
        if node in vertex_centrality_all:
            vertex_centrality_q[node] = vertex_centrality_all[node]
        else:
            vertex_centrality_q[node] = 0  # 若不存在，默认中心性为0

    edge_centrality_q = {}
    for u, v in Q.edges():
        if (u, v) in edge_centrality_all:
            edge_centrality_q[(u, v)] = edge_centrality_all[(u, v)]
        elif (v, u) in edge_centrality_all:  # 无向边
            edge_centrality_q[(u, v)] = edge_centrality_all[(v, u)]
        else:
            edge_centrality_q[(u, v)] = 0  # 若不存在，默认中心性为0

    return vertex_centrality_q, edge_centrality_q

In [ ]:
def entropy_weight_method(data_dict):
    """
    使用熵值法计算指标的权重（适用于交易频率和成功率）

    参数：
    - data_dict: dict，格式为 {item_id: {'freq': ..., 'succ_rate': ...}}

    返回：
    - weight_dict: dict，格式为 {item_id: 权重值}
    """
    if not data_dict:
        return {}

    df = pd.DataFrame.from_dict(data_dict, orient='index')  # index=item_id, columns=['freq', 'succ_rate']
    
    # 若全部为 0，则避免除0错误
    if (df.sum().sum() == 0):
        return {item: 0 for item in data_dict}
    
    # 正向指标标准化（极小-极大标准化）
    norm_df = (df - df.min()) / (df.max() - df.min() + 1e-12)  # 加小常数防止除以0
    
    # 计算每列的熵
    eps = 1e-12
    P = norm_df / (norm_df.sum(axis=0) + eps)
    entropy = -np.sum(P * np.log(P + eps), axis=0) / np.log(len(df))

    # 计算冗余度与权重
    redundancy = 1 - entropy
    weights = redundancy / np.sum(redundancy)

    # 计算每个 item 的加权得分作为最终权重
    final_scores = norm_df @ weights
    weight_dict = dict(zip(df.index, final_scores))

    return weight_dict

In [ ]:
def compute_vertex_entropy_weights(vertex_freq_dict, vertex_succ_dict):
    """
    对顶点进行熵值加权计算
    输入:
        vertex_freq_dict: {node: freq}
        vertex_succ_dict: {node: succ_rate}
    输出:
        {node: weight}
    """
    data = {}
    for node in set(vertex_freq_dict) | set(vertex_succ_dict):
        data[node] = {
            'freq': vertex_freq_dict.get(node, 0),
            'succ_rate': vertex_succ_dict.get(node, 0)
        }
    return entropy_weight_method(data)


def compute_edge_entropy_weights(edge_freq_dict, edge_succ_dict):
    """
    对边进行熵值加权计算
    输入:
        edge_freq_dict: {(u, v): freq}
        edge_succ_dict: {(u, v): succ_rate}
    输出:
        {(u, v): weight}
    """
    data = {}
    for edge in set(edge_freq_dict) | set(edge_succ_dict):
        data[edge] = {
            'freq': edge_freq_dict.get(edge, 0),
            'succ_rate': edge_succ_dict.get(edge, 0)
        }
    return entropy_weight_method(data)


In [ ]:
def update_prices_advanced(
    query_graph,
    vertex_price_list,
    edge_price_list,
    new_price,
    centrality_file,
    full_transaction_history,
    centrality_weight=0.2
):
    """
    优化后的价格更新函数：结合熵值法和中心性权重调整价格（动态计算频率与成功率）

    :param query_graph: 查询图（networkx.Graph）
    :param vertex_price_list: 顶点价格字典 {name: {'cost': c, 'price': p}}
    :param edge_price_list: 边价格字典 {(u, v): {'weight': c, 'price': p}}
    :param new_price: 目标总价格
    :param centrality_file: pickle 文件路径，包含所有顶点和边的中心性值
    :param full_transaction_history: 历史交易记录列表，每条记录是 {'graph': G, 'price': float, 'success': bool}
    :param centrality_weight: 中心性权重（经验设置）
    """
    def normalize(d):
        total = sum(d.values()) or 1e-6
        return {k: v / total for k, v in d.items()}

    # === 中心性提取 ===
    vertex_centrality_all, edge_centrality_all = get_centrality_for_query(query_graph, centrality_file)

    used_vertices = set()
    used_edges = set()
    total_cost = 0

    # === 获取图中实际使用的顶点和边 ===
    for node in query_graph.nodes:
        name = query_graph.nodes[node].get('name', node)
        if name in vertex_price_list:
            used_vertices.add(name)
            total_cost += vertex_price_list[name]['cost']

    for u, v in query_graph.edges:
        name_u = query_graph.nodes[u].get('name', u)
        name_v = query_graph.nodes[v].get('name', v)
        edge_key = tuple(sorted((name_u, name_v)))
        if edge_key in edge_price_list:
            used_edges.add(edge_key)
            total_cost += edge_price_list[edge_key]['weight']

    # === fallback：按成本等比缩放 ===
    if total_cost >= new_price:
        adjustment_ratio = new_price / total_cost
        for name in used_vertices:
            cost = vertex_price_list[name]['cost']
            vertex_price_list[name]['price'] = round(cost * adjustment_ratio, 4)
        for edge in used_edges:
            weight = edge_price_list[edge]['weight']
            edge_price_list[edge]['price'] = round(weight * adjustment_ratio, 4)
        return vertex_price_list, edge_price_list

    # === 动态计算频率与成功率 ===
    vertex_freq_dict, vertex_succ_dict, edge_freq_dict, edge_succ_dict = compute_frequency_and_success_rate(
        query_graph, full_transaction_history
    )

    # === 熵值法权重 ===
    v_weights = compute_vertex_entropy_weights(
        {k: vertex_freq_dict.get(k, 0) for k in used_vertices},
        {k: vertex_succ_dict.get(k, 0) for k in used_vertices}
    )
    e_weights = compute_edge_entropy_weights(
        {k: edge_freq_dict.get(k, 0) for k in used_edges},
        {k: edge_succ_dict.get(k, 0) for k in used_edges}
    )

    # === 提取中心性 ===
    v_center = {k: vertex_centrality_all.get(k, 0) for k in used_vertices}
    e_center = {k: edge_centrality_all.get(k, 0) for k in used_edges}

    # === 归一化 ===
    v_weights_norm = normalize(v_weights)
    e_weights_norm = normalize(e_weights)
    v_center_norm = normalize(v_center)
    e_center_norm = normalize(e_center)

    # === 组合权重 ===
    v_combined = {
        k: (1 - centrality_weight) * v_weights_norm.get(k, 0) + centrality_weight * v_center_norm.get(k, 0)
        for k in used_vertices
    }
    e_combined = {
        k: (1 - centrality_weight) * e_weights_norm.get(k, 0) + centrality_weight * e_center_norm.get(k, 0)
        for k in used_edges
    }

    # === 分配价格 ===
    remaining = new_price - total_cost
    total_combined = sum(v_combined.values()) + sum(e_combined.values()) or 1e-6
    v_ratio = {k: v / total_combined for k, v in v_combined.items()}
    e_ratio = {k: v / total_combined for k, v in e_combined.items()}

    for name in used_vertices:
        base_cost = vertex_price_list[name]['cost']
        alloc = remaining * v_ratio.get(name, 0)
        vertex_price_list[name]['price'] = round(base_cost + alloc, 4)

    for edge in used_edges:
        base_cost = edge_price_list[edge]['weight']
        alloc = remaining * e_ratio.get(edge, 0)
        edge_price_list[edge]['price'] = round(base_cost + alloc, 4)

    return vertex_price_list, edge_price_list

In [ ]:
def update_prices_advanced(
    query_graph,
    vertex_price_list,
    edge_price_list,
    vertex_freq_dict,
    vertex_succ_dict,
    edge_freq_dict,
    edge_succ_dict,
    new_price,
    centrality_file,
    centrality_weight=0.2
):
    """
    优化后的价格更新函数：结合熵值法和中心性权重调整价格（中心性自动提取）

    :param query_graph: 查询图
    :param vertex_price_list: 顶点价格字典 {name: {'cost': c, 'price': p}}
    :param edge_price_list: 边价格字典 {(u, v): {'weight': c, 'price': p}}
    :param vertex_freq_dict / vertex_succ_dict: 顶点频率/成功率字典
    :param edge_freq_dict / edge_succ_dict: 边频率/成功率字典
    :param new_price: 目标总价格
    :param centrality_file: pickle 文件路径，包含所有顶点和边的中心性值
    :param centrality_weight: 中心性权重（主观经验权重）
    """

    def normalize(d):
        total = sum(d.values()) or 1e-6
        return {k: v / total for k, v in d.items()}

    # === 中心性提取 ===
    vertex_centrality_all, edge_centrality_all = get_centrality_for_query(query_graph, centrality_file)

    used_vertices = set()
    used_edges = set()
    total_cost = 0

    # === 获取图中实际使用的顶点和边 ===
    for node in query_graph.nodes:
        name = query_graph.nodes[node].get('name', node)
        if name in vertex_price_list:
            used_vertices.add(name)
            total_cost += vertex_price_list[name]['cost']

    for u, v in query_graph.edges:
        name_u = query_graph.nodes[u].get('name', u)
        name_v = query_graph.nodes[v].get('name', v)
        edge_key = tuple(sorted((name_u, name_v)))
        if edge_key in edge_price_list:
            used_edges.add(edge_key)
            total_cost += edge_price_list[edge_key]['weight']

    # === fallback：按成本等比缩放 ===
    if total_cost >= new_price:
        adjustment_ratio = new_price / total_cost
        for name in used_vertices:
            cost = vertex_price_list[name]['cost']
            vertex_price_list[name]['price'] = round(cost * adjustment_ratio, 4)
        for edge in used_edges:
            weight = edge_price_list[edge]['weight']
            edge_price_list[edge]['price'] = round(weight * adjustment_ratio, 4)
        return vertex_price_list, edge_price_list

    # === 熵值法权重 ===
    v_weights = compute_vertex_entropy_weights(
        {k: vertex_freq_dict.get(k, 0) for k in used_vertices},
        {k: vertex_succ_dict.get(k, 0) for k in used_vertices}
    )
    e_weights = compute_edge_entropy_weights(
        {k: edge_freq_dict.get(k, 0) for k in used_edges},
        {k: edge_succ_dict.get(k, 0) for k in used_edges}
    )

    # === 提取中心性 ===
    v_center = {k: vertex_centrality_all.get(k, 0) for k in used_vertices}
    e_center = {k: edge_centrality_all.get(k, 0) for k in used_edges}

    # === 归一化 ===
    v_weights_norm = normalize(v_weights)
    e_weights_norm = normalize(e_weights)
    v_center_norm = normalize(v_center)
    e_center_norm = normalize(e_center)

    # === 组合权重 ===
    v_combined = {
        k: (1 - centrality_weight) * v_weights_norm.get(k, 0) + centrality_weight * v_center_norm.get(k, 0)
        for k in used_vertices
    }
    e_combined = {
        k: (1 - centrality_weight) * e_weights_norm.get(k, 0) + centrality_weight * e_center_norm.get(k, 0)
        for k in used_edges
    }

    # === 分配价格 ===
    remaining = new_price - total_cost
    total_combined = sum(v_combined.values()) + sum(e_combined.values()) or 1e-6
    v_ratio = {k: v / total_combined for k, v in v_combined.items()}
    e_ratio = {k: v / total_combined for k, v in e_combined.items()}

    for name in used_vertices:
        base_cost = vertex_price_list[name]['cost']
        alloc = remaining * v_ratio.get(name, 0)
        vertex_price_list[name]['price'] = round(base_cost + alloc, 4)

    for edge in used_edges:
        base_cost = edge_price_list[edge]['weight']
        alloc = remaining * e_ratio.get(edge, 0)
        edge_price_list[edge]['price'] = round(base_cost + alloc, 4)

    return vertex_price_list, edge_price_list


In [ ]:
def process_transactions(
    queries,
    subgraphs,
    vertex_price_list,
    edge_price_list,
    transaction_manager,
    full_transaction,
    plist_buyer,
    evaluator,
    start_time,
    xpricing=1,
    centrality_file=None
):
    evaluation_intervals = {10000, 30000, 50000, 80000, 120000, 150000}
    results = {}

    for i, Q in enumerate(queries, start=1):
        expected_price = evaluator.generate_expected_price(Q)

        # 更新或添加预期价格记录
        found = False
        for p_b in plist_buyer:
            if gequal(Q, p_b[0]):
                p_b[1] = expected_price
                found = True
                break
        if not found:
            plist_buyer.append([Q, expected_price])

        # --- Pricing Section ---
        if xpricing == 0:
            # 子图匹配定价
            price, _ = subisomorphic_pricing(Q, subgraphs, vertex_price_list, edge_price_list)
            success = expected_price >= price
            print(f'第{i}次交易：{"成功" if success else "失败"} - Q: {Q}, 价格: {price}, 预期价格: {expected_price}')
        else:
            record = transaction_manager.get_summary(Q)

            if record:
                cost = computecost_G(Q, vertex_price_list, edge_price_list)
                price = calculate_query_price(Q, vertex_price_list, edge_price_list)

                sp_max = record['max_success_price']
                fp_min = record['min_fail_price']
                success_count = record['success_count']
                fail_count = record['fail_count']

                new_price = price

                if expected_price < price:
                    success = False
                    print(f'第{i}次交易：失败 - Q: {Q}, 价格: {price}, 预期价格: {expected_price}')
                    if success_count > 0:
                        new_price = (sp_max + price) / 2
                    else:
                        diff = price - cost
                        if diff > xpricing:
                            new_price = price - xpricing
                        elif diff > (xpricing / 2):
                            new_price = price - xpricing / 2
                        elif diff > 1:
                            new_price = price - 1
                        else:
                            new_price = cost + 1
                else:
                    success = True
                    print(f'第{i}次交易：成功 - Q: {Q}, 价格: {price}, 预期价格: {expected_price}')
                    if fail_count > 0 and fp_min != float('inf'):
                        new_price = (fp_min + price) / 2
                    else:
                        new_price = price + xpricing

                if not isinstance(new_price, (int, float)) or not (new_price < float('inf')):
                    print("Error: 非法 new_price，跳过 Q")
                    continue

                if new_price != price:
                    vertex_freq_dict, edge_freq_dict, vertex_succ_dict, edge_succ_dict = compute_frequency_and_success_rate(Q, full_transaction.get_history())
                    vertex_centrality_dict, edge_centrality_dict = calculate_centrality(Q, centrality_file)
                    vertex_price_list, edge_price_list = update_prices_advanced(
                        Q,
                        vertex_price_list,
                        edge_price_list,
                        vertex_freq_dict,
                        vertex_succ_dict,
                        edge_freq_dict,
                        edge_succ_dict,
                        vertex_centrality_dict,
                        edge_centrality_dict,
                        new_price
                    )

                price = new_price

            else:
                # 无历史记录：先进行子图匹配定价
                price, _ = subisomorphic_pricing(Q, subgraphs, vertex_price_list, edge_price_list)
                success = expected_price >= price
                print(f'第{i}次交易：{"成功" if success else "失败"} - Q: {Q}, 价格: {price}, 预期价格: {expected_price}')

                # 无历史记录也进行价格更新
                new_price = expected_price if success else price
                vertex_freq_dict, edge_freq_dict, vertex_succ_dict, edge_succ_dict = calculate_transaction_metrics(Q, full_transaction.get_history())
                vertex_centrality_dict, edge_centrality_dict = calculate_centrality(Q, centrality_file)
                vertex_price_list, edge_price_list = update_prices_advanced(
                    Q,
                    vertex_price_list,
                    edge_price_list,
                    vertex_freq_dict,
                    vertex_succ_dict,
                    edge_freq_dict,
                    edge_succ_dict,
                    vertex_centrality_dict,
                    edge_centrality_dict,
                    new_price
                )

        # 记录交易
        transaction_manager.add_transaction(Q, price, success)
        full_transaction.add_transaction(Q, price, success)

        # 定期评估并保存结果
        if i in evaluation_intervals:
            e_time = time.time() - start_time
            metrics = evaluator.evaluate_transactions(full_transaction.get_history())
            results = {
                "success_ratio": metrics[0],
                "avg_regret": metrics[1],
                "avg_price_deviation": metrics[2],
                "avg_sdiff": metrics[3],
                "avg_diff": metrics[4],
                "avg_per_diff": metrics[5],
                "customer_avg_per_diff": metrics[6],
                "time": e_time
            }
            evaluator.save_evaluation_results(i, "TEST_x0_test_evaluation_results.txt", results)

    # 最终评估
    e_time = time.time() - start_time
    metrics = evaluator.evaluate_transactions(full_transaction.get_history())
    results = {
        "success_ratio": metrics[0],
        "avg_regret": metrics[1],
        "avg_price_deviation": metrics[2],
        "avg_sdiff": metrics[3],
        "avg_diff": metrics[4],
        "avg_per_diff": metrics[5],
        "customer_avg_per_diff": metrics[6],
        "time": e_time
    }
    print(results)
    evaluator.save_evaluation_results(i, "TEST_x0_test_evaluation_results.txt", results)


In [ ]:
#熵值法
#1.归一化所有属性值
def min_max_normalize(data_dict):
    values = list(data_dict.values())
    min_val, max_val = min(values), max(values)
    if max_val == min_val:
        return {k: 0.0 for k in data_dict}  # 防止除0
    return {k: (v - min_val) / (max_val - min_val) for k, v in data_dict.items()}

In [ ]:
#熵值法
#2.对交易频率和成功率使用熵值法确定权重
def entropy_weight_method(attributes_list):
    """
    attributes_list: List[Dict[id, value]]，每项代表一个属性的归一化结果

    返回：
    - List[float]，每个属性的权重
    """
    ids = attributes_list[0].keys()
    data = np.array([[attr[i] for i in ids] for attr in attributes_list], dtype=float)
    eps = 1e-10

    # 第一步：归一化
    P = data / (data.sum(axis=1, keepdims=True) + eps)

    # 第二步：计算熵值
    E = -np.sum(P * np.log(P + eps), axis=1) / np.log(len(ids))

    # 第三步：计算权重
    d = 1 - E
    weights = d / d.sum()
    return weights.tolist()

In [ ]:
# Step 1: 归一化
freq_norm = min_max_normalize(vertex_tf)
tsr_norm = min_max_normalize(vertex_tsr)

# Step 2: 熵值法动态计算 w1, w2
w1, w2 = entropy_weight_method([freq_norm, tsr_norm])

# Step 3: 设置中心性静态权重 w3
w3 = 0.3  # 可设为 0.2～0.4，根据经验调整

In [ ]:
score_dict = {}
for node in query_graph.nodes:
    score_dict[node] = (
        w1 * freq_norm.get(node, 0.0) +
        w2 * tsr_norm.get(node, 0.0) +
        w3 * centrality_norm.get(node, 0.0)
    )
